In [1]:
import pandas as pd
import numpy as np
from thefuzz import process

In [3]:
df_listino = pd.read_excel('data_clean/art_costi.xlsx')
df_ddt = pd.read_excel('data_clean/ddt_righe.xlsx')

In [4]:
IDs_listino_univoci = df_listino['Codice_articolo'].unique().tolist()
n_IDs_listino = len(IDs_listino_univoci)
IDs_ddt = df_ddt['Codice_articolo'].tolist()
IDs_ddt_univoci = df_ddt['Codice_articolo'].unique().tolist()
n_IDs_ddt = df_ddt['Codice_articolo'].nunique()

In [5]:
IDs_ddt_not_in_listino = [articolo for articolo in IDs_ddt if articolo not in IDs_listino_univoci]
IDs_ddt_not_in_listino_univoci = list(set(IDs_ddt_not_in_listino))
n_IDs_ddt_not_in_listino = len(IDs_ddt_not_in_listino)
n_IDs_ddt_not_in_listino_univoci = len(IDs_ddt_not_in_listino_univoci)

IDs_listino_not_in_ddt_univoci = [articolo for articolo in IDs_listino_univoci if articolo not in IDs_ddt_univoci]
n_IDs_listino_not_in_ddt = len(IDs_listino_not_in_ddt_univoci)

print('Numero articoli nel listino: ', n_IDs_listino)
print('Numero articoli nei DDT: ', n_IDs_ddt)
print('Numero articoli nei DDT non presenti nel listino: ', n_IDs_ddt_not_in_listino)
print('Numero articoli nei DDT non presenti nel listino (univoci): ', n_IDs_ddt_not_in_listino_univoci)
print('Numero articoli nel listino non presenti nei DDT (univoci): ', n_IDs_listino_not_in_ddt)
print('Articoli nei DDT non presenti nel listino (univoci): ', IDs_ddt_not_in_listino_univoci)
print('Articoli nel listino non presenti nei DDT (univoci): ', IDs_listino_not_in_ddt_univoci)

Numero articoli nel listino:  1311
Numero articoli nei DDT:  1168
Numero articoli nei DDT non presenti nel listino:  12258
Numero articoli nei DDT non presenti nel listino (univoci):  292
Numero articoli nel listino non presenti nei DDT (univoci):  435
Articoli nei DDT non presenti nel listino (univoci):  ['017.PM105.04', '017.PM117AN', 'KCL.MONT', '038.STC44.1F145.01', '017.TD.COM.002.985', '091.NW2605AN', '017.IMB03', '091.NW2604AN', '038.STC58.2E170.01', '028.KCP.SX', '038.STC44.1A120.01', '028.266VFOR', 'COM.AUTS', '091.NW2606AN', '017.PM458', '017.IMB05', '017.PM151', '028.259VFOR', 'FINISH.01', '028.KANT13', '038.STC44.1D120.01', '028.021LMH', '027.017ACZI', '091.NW2602AN', '017.IMB13', '017.IMB16', '017.PM051.25', 'COM.IMPV', '027.104MDF', '090.ANTATR.13', '017.TD.ACC.023', '017.IMB19', 'REC.SPE', 'COM.SED', '017.PM105.24', '092.NW2533PL', '052.KIT.CTS.SPS.007', '091.N1042AN', '017.TD.COM.002.1025', '092.ANTATR.03', '017.IMB06', '091.NW2612AN', '028.365VFOR', '090.ANTATR.05', '0

In [8]:
IDs_ddt_not_in_listino_univoci[0]

'028.KANT13'

In [8]:
for i in range(0, n_IDs_ddt_not_in_listino_univoci):
    print('ID DDT: ', IDs_ddt_not_in_listino_univoci[i])
    print('ID più simile nel listino: ', process.extract(IDs_ddt_not_in_listino_univoci[i], IDs_listino_univoci, limit=1)[0])
    print('\n')

ID DDT:  017.PM105.04
ID più simile nel listino:  ('017.PM105.01', 92)


ID DDT:  017.PM117AN
ID più simile nel listino:  ('017.PM117', 90)


ID DDT:  KCL.MONT
ID più simile nel listino:  ('052.ACC.MOB.001', 53)


ID DDT:  038.STC44.1F145.01
ID più simile nel listino:  ('017.PM103.01', 86)


ID DDT:  017.TD.COM.002.985
ID più simile nel listino:  ('017.PM001', 86)


ID DDT:  091.NW2605AN
ID più simile nel listino:  ('091.NW2605', 91)


ID DDT:  017.IMB03
ID più simile nel listino:  ('017.GUI.MED.01', 86)


ID DDT:  091.NW2604AN
ID più simile nel listino:  ('091.NW2604', 91)


ID DDT:  038.STC58.2E170.01
ID più simile nel listino:  ('017.PM103.01', 86)


ID DDT:  028.KCP.SX
ID più simile nel listino:  ('052.MP.CHSL.SX', 63)


ID DDT:  038.STC44.1A120.01
ID più simile nel listino:  ('017.PM103.01', 86)


ID DDT:  028.266VFOR
ID più simile nel listino:  ('028.220', 86)


ID DDT:  COM.AUTS
ID più simile nel listino:  ('017.TD.COM.001.1030', 86)


ID DDT:  091.NW2606AN
ID più simile nel lis